# Agents

In this notebook we'll explore agents and how to use them in [LangChain](https://python.langchain.com/en/latest/index.html#).

List of main relevant papers:
* [Karpas et al. (2022)](https://arxiv.org/pdf/2205.00445.pdf). MRKL Systems: A modular, neuro-symbolic architecture that combines large language
models, external knowledge sources and discrete reasoning
* [Shen et al. (2023)](https://arxiv.org/pdf/2303.17580.pdf). HuggingGPT: Solving AI Tasks with ChatGPT and its Friends in Hugging Face.
* [Wang et al. (2023)](https://arxiv.org/pdf/2305.04091.pdf). Plan-and-Solve Prompting: Improving Zero-Shot Chain-of-Thought Reasoning by Large Language Models
* [Press et al. (2022)](https://arxiv.org/pdf/2210.03350.pdf). Measuring and narrowing the composability gap in language models.
* [Yao et al. (2023)](https://arxiv.org/pdf/2210.03629.pdf). ReAct: synergizing reasoning and acting in language models.

List of main relevant videos:
* [Briggs (2023)](https://www.youtube.com/watch?v=jSP-gSEyVeI). LangChain Agents Deep Dive with GPT 3.5.

----

**Note:** This work is an extended version of the [tutorial by James Briggs](https://github.com/pinecone-io/examples/blob/master/generation/langchain/handbook/06-langchain-agents.ipynb)

----

## 1 - Introduction

In recent years, large language models have revolutionized natural language processing tasks, showcasing remarkable advancements in generating human-like text, answering questions, and even engaging in intelligent conversations. These models, such as GPT, PaLM, LLaMA, etc. have astounded researchers and practitioners alike with their ability to comprehend and produce coherent and contextually appropriate responses. However, despite their unprecedented capabilities, these state-of-the-art models still possess limitations that hinder their full potential.

1. **Lack of access to current information.** Certain data constantly change. For instance, the
exchange rate between the dollar and the Moroccan Dirham, current COVID numbers, the stock price of AAPL, the weather in Vancouver or even the current date. It is impossible, by their design, for pretrained language models to keep up with this dynamic information

2. **Lack of access to proprietary information sources.** As an important special case of 1, the models don’t have access to proprietary information, such as the client roster in a company’s database or the state of an online game.

3. **Lack of probabilistic reasoning.** Probabilistic reasoning is beyond the reach of the neural approach (or it would require even more massive models that we cannot easily train/work with) and requires a dedicated reasoning process.

4. **Lack of arithmetic reasoning.** With GPT-4, we can see a serious improvement, but it still lacks precision, especially when dealing with float numbers.

5. **Model explosion.** As the demand for larger models grows, it is crucial to balance their capabilities with the cost and environmental impact. In this context, exploring alternative model architectures and training algorithms can promote energy efficiency without compromising performance.

A potential solution to mitigate these limiatations is to enhance the utility of the language model with complementary tools. These tools can provide new functionalities and assist in bias detection and mitigation, content filtering, controlled generation, etc. 

To the best of my knowledge, this idea was first introducedd by [Karpas et al. (2022)](https://arxiv.org/pdf/2205.00445.pdf), which defined a flexible architecture called *MRKL* (i.e., *Modular Reasoning, Knowledge and Language*). A MRKL system consists of an extendable set of modules and a "router" (i.e., an LLM) that routes every incoming natural language input to a module that can best respond to the input. 

<table>
    <tr>
        <td><img src="images_2/MRKL.png" width="300"/></td>
    </tr>
</table>

This idea has been recently extended by [Shen et al. (2023)](), where models are automatically downloaded from HuggingFace, instead of using a pre-defined one. In this paper, an LLM acts as the "controller". The LLM first plans a list of tasks based on the user request and then assigns expert models to each task. After the experts execute the tasks, the LLM collects the results and responds to the user.

<table>
    <tr>
        <td><img src="images_2/HuggingGPT.png" width="700"/></td>
    </tr>
</table>

## 2 - Langchain's agents

Some applications will require not just a predetermined chain of calls to LLMs/other tools, but potentially an unknown chain that depends on the user’s input. In these types of chains, there is a “agent” which has access to a suite of tools. Depending on the user input, the agent can then decide which, if any, of these tools to call. The utility of [**LangChain**](https://python.langchain.com/en/latest/index.html#) becomes apparent in this context, as it provides a versatile programming framework for constructing such systems.

At the moment, there are two main types of agents in LangChain:
* [**Action Agents**](https://python.langchain.com/en/latest/modules/agents.html#action-agents): these agents decide an action to take and take that action one step at a time
* [**Plan-and-Execute Agents**](https://python.langchain.com/en/latest/modules/agents/plan_and_execute.html): these agents first decide a plan of actions to take, and then execute those actions one at a time.

----

**When should you use each one?** 

Action Agents are more conventional, and good for small tasks. For more complex or long running tasks, the initial planning step helps to maintain long term objectives and focus. However, that comes at the expense of generally more calls and higher latency. These two agents are also not mutually exclusive - in fact, it is often best to have an Action Agent be in change of the execution for the Plan-and-Execute agent.

----

#### Create language model

The basis of our agents is the multi-purpose language model. In this case, we are going to consider [OpenAI's `text-davinci-003` model](https://platform.openai.com/docs/models/gpt-3-5) (i.e., a version of GPT 3.5).

In [1]:
import openai
import os
from langchain.llms import AzureOpenAI

openai.api_type = "azure"
openai.api_base = "https://gpt3tests.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.environ["OPENAI_API_KEY"]

engine = "Davinci003"
max_tokens = 1000

language_model = AzureOpenAI(deployment_name=engine)
language_model.openai_api_key = openai.api_key
language_model.openai_api_base = openai.api_base 
language_model.max_tokens = max_tokens

We are going to define a custom function to count the number of tokens of a query

In [2]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

#### Create example Database

We will use the agents to interact with a small sample database of stocks. We will not dive into the details because this is just a dummy tool we will build for illustrative purposes.


In [23]:
from sqlalchemy import MetaData
from sqlalchemy import Column, Integer, String, Table, Date, Float
from sqlalchemy import create_engine
from sqlalchemy import insert
from datetime import datetime

def insert_obs(obs, table):
    stmt = insert(table).values(
    obs_id=obs[0], 
    stock_ticker=obs[1], 
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [24]:
metadata_obj = MetaData()

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),    
)

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

for obs in observations:
    insert_obs(obs, stocks)

Once we have inserted all of the observations in the Database, we need to create a wrapper that allows Langchain to properly access it. This can be done with the `SQLDatabase` class.

In [25]:
from langchain.sql_database import SQLDatabase

db = SQLDatabase(engine)

Finally, we will create a chain and a custom tool from it. To create a custom tool we need only three inputs:
* `name`: the name of the tool (sent to the llm as context)
* `func`: the function that will be applied on the LLM's request
* `description`: the description of the tool, what it should be used for (sent to the llm as context)

In [26]:
from langchain.chains import SQLDatabaseChain
from langchain.agents import Tool

sql_chain = SQLDatabaseChain.from_llm(llm=language_model, db=db, verbose=True)
sql_tool = Tool(
    name='Stock DB',
    func=sql_chain.run,
    description="Useful for when you need to answer questions about stocks " \
                "and their prices."
)

## 3 - Action agents

Action agents are designed to solve small tasks with a combination of a LLM and various tools.

**High level pseudocode of action agents:**

1. Some user input is received.
2. The agent decides which tool - if any - to use, and what the input to that tool should be.
3. That tool is then called with that tool input, and an observation is recorded (this is just the output of calling that tool with that tool input).
4. That history of tool, tool input, and observation is passed back into the agent, and it decides what step to take next.
5. This is repeated until the agent decides it no longer needs to use a tool, and then it responds directly to the user.

**Typical way to construct an action agent:**

* [**PromptTemplate:**](https://python.langchain.com/en/latest/modules/prompts/prompt_templates.html) this is responsible for taking the user input and previous steps and constructing a prompt to send to the language model
* [**Language Model:**](https://python.langchain.com/en/latest/modules/models/llms.html) this takes the prompt constructed by the PromptTemplate and returns some output
* [**Output Parser:**](https://python.langchain.com/en/latest/modules/prompts/output_parsers.html) this takes the output of the Language Model and parses it into an AgentAction or AgentFinish object.

**The different abstractions involved in action agents:**

* **Agent:** this is where the logic of the application lives. Agents expose an interface that takes in user input along with a list of previous steps the agent has taken, and returns either an AgentAction or AgentFinish.
    *  *AgentAction* corresponds to the tool to use and the input to that tool.
    * *AgentFinish* means the agent is done, and has information around what to return to the user.
* [**Tools:**](https://python.langchain.com/en/latest/modules/agents/tools.html) these are the actions an agent can take. What tools you give an agent highly depend on what you want the agent to do.
* [**Toolkits:**](https://python.langchain.com/en/latest/modules/agents/toolkits.html) these are groups of tools designed for a specific use case. For example, [in order for an agent to interact with a SQL database in the best way it may need access to one tool to execute queries and another tool to inspect tables](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/sql_database.html).


### 3.1 - Zero-shot ReAct (MRKL)

As the name suggests, we will use this agent to perform 'zero shot' tasks on the input. That means that we will not have several, interdependent interactions but only one. In other words, this agent will have no memory.

This agent uses the [ReAct framework](https://arxiv.org/pdf/2210.03629.pdf) to determine which tool to use based solely on the tool's description. Any number of tools can be provided. This agent requires that a description is provided for each tool.

* We first need to initialize the tools we'll be using in this example. [Here we will want our agent to do math so we will use **llm-math**](https://python.langchain.com/en/latest/modules/chains/examples/llm_math.html), the tool we have for solving math problems. [This is one of the several tools we can load with the **load_tools()** function.](https://python.langchain.com/en/latest/modules/agents/tools.html#tools)

* Then, to be able to search across our stock prices, we will also use the custom tool we built beforehand with the sql data ('Stock DB'). We will append this tool to our list of tools.

In [27]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"], 
    llm=language_model
)
tools.append(sql_tool)

Now we are ready to initialize the agent! We will use verbose in True so we can see what is our agent's 'thinking' process.

----

**Important Note:** When interacting with agents it is really important to set the max_iterations parameters because agents can get stuck in infinite loops that consume plenty of tokens. The default value is 15 to allow for many tools and complex reasoning but for most applications you should keep it much lower.

----

In [28]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description", 
    tools=tools, 
    llm=language_model,
    verbose=True,
    max_iterations=3,
)

In [29]:
result = count_tokens(
    zero_shot_agent, 
    "What is the multiplication of the ratio between stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th?"
)



> Entering new AgentExecutor chain...
 I need to compare the ratio between stock prices for 'ABC' and 'XYZ' between January 3rd and January 4th.
Action: Stock DB
Action Input: Obtain ratio of stock prices for 'ABC' and 'XYZ' on January 3rd and January 4th.

> Entering new SQLDatabaseChain chain...
Obtain ratio of stock prices for 'ABC' and 'XYZ' on January 3rd and January 4th.
SQLQuery:SELECT (S1.price/S2.price) AS ratio FROM stocks AS S1 INNER JOIN stocks AS S2 ON S1.stock_ticker='ABC' AND S2.stock_ticker='XYZ' WHERE (S1.date='2023-01-03' AND S2.date='2023-01-04') LIMIT 5;
SQLResult: [(0.2918238993710692,)]
Answer:The ratio of stock prices for 'ABC' and 'XYZ' on January 3rd and January 4th is 0.2918238993710692.
> Finished chain.

Observation: The ratio of stock prices for 'ABC' and 'XYZ' on January 3rd and January 4th is 0.2918238993710692.
Thought: I need to multiply the two ratios to get the final answer.
Action: Calculator
Action Input: Multiply 0.2918238993710692 and 0.29182389

Let's take a look at the prompt:

In [30]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.
Stock DB: Useful for when you need to answer questions about stocks and their prices.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, Stock DB]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


The agent follows the MRKL approach and has the ability to "reason" on how to best use tools to solve our query and can combine them in intelligent ways with just a brief description of each of them.

Finally, let's pay attention to the **agent_scratchpad**. What is that? Well, that is where we will be appending every thought or action that the agent has already performed. In this way, at each point in time, the agent will know what it has found out and will be able to continue its thought process. In other words, after using a tool it adds its thoughts and observations to the scratchpad and picks up from there.

### 3.2 - Conversational ReAct (MRKL + buffer memory)

The zero shot agent is really interesting but, as we said before, **it has no memory**. What if we want an assistant that remembers things we have talked about and can also reason about them and use tools? For that we have the conversational react agent.

The memory type being used here is a simple buffer memory to allow us to remember previous steps in the reasoning chain. [For more information on memory see this link.](https://python.langchain.com/en/latest/modules/memory.html)

In [31]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=language_model,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

As we can see below, the prompt is similar but it includes a great prelude of instructions that make it an effective assistant as well + a spot for including the chat history from the memory component:

In [32]:
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [33]:
result = count_tokens(
    conversational_agent, 
    "Please provide me the stock prices for ABC on January the 1st"
)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Stock DB
Action Input: ABC, January 1st

> Entering new SQLDatabaseChain chain...
ABC, January 1st
SQLQuery:SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-01'
SQLResult: [(200.0,)]
Answer:ABC stock price on January 1st is 200.0
> Finished chain.

Observation: ABC stock price on January 1st is 200.0
Thought: Do I need to use a tool? No
AI: The stock price of ABC on January 1st was 200.0.

> Finished chain.
Spent a total of 1827 tokens


In [34]:
result = count_tokens(
    conversational_agent, 
    "What is that to the power of 0.43?"
)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 200.0 to the power of 0.43
Observation: Answer: 9.759844497019593
Thought: Do I need to use a tool? No
AI: The answer is 9.759844497019593.

> Finished chain.
Spent a total of 1240 tokens


As we can see, it stores previous information in memory so we can ask a chain of questions.

The conversational assistant is designed for conversations but faces some difficulties when it comes to integrating a complex set of tools. Let's observe the outcome when we ask it to answer the same question we presented to our previous agent:

In [35]:
result = count_tokens(
    conversational_agent, 
    "What is the multiplication of the ratio of the prices of stocks 'ABC' and 'XYZ' in January 3rd and the ratio of the same prices of the same stocks in January the 4th?"
)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Stock DB
Action Input: Ratio of prices of stocks ABC and XYZ in January 3rd and the ratio of the same prices of the same stocks in January 4th

> Entering new SQLDatabaseChain chain...
Ratio of prices of stocks ABC and XYZ in January 3rd and the ratio of the same prices of the same stocks in January 4th
SQLQuery:SELECT stock_ticker, (price / LAG(price, 1) OVER (PARTITION BY stock_ticker ORDER BY date DESC)) AS price_ratio
FROM stocks
WHERE (date = '2023-01-03' OR date = '2023-01-04') AND stock_ticker IN ('ABC', 'XYZ')
SQLResult: [('ABC', None), ('ABC', 1.031111111111111), ('XYZ', None), ('XYZ', 1.0037735849056604)]
Answer:The ratio of prices of stocks ABC and XYZ in January 3rd is 1.031111111111111, and the ratio of the same prices of the same stocks in January 4th is 1.0037735849056604.
> Finished chain.

Observation: The ratio of prices of stocks ABC and XYZ in January 3rd is 1.031111111111111, and

In this case, it usually gets the answer wrong by trying to solve it all only with SQL. I have also seen it get the SQL wrong.

-----

**Note:** I am having doubts with the default prompt. It may require some tweaking to improve it. Additionally, it could use a fine-tuned SQL tool

-----

### 3.3 - Docstore ReAct

This type of agent is similar to the ones we have seen so far but it includes the interaction with a docstore. It will have two and only two tools at its disposal: 

* **Search**. The agent uses this tool to bring up a relevant article.
* **LookUp**. The agent tries to find the right piece of information in the article.

In [42]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

docstore_agent = initialize_agent(
    tools, 
    language_model, 
    agent="react-docstore", 
    verbose=True,
    max_iterations=3
)

The associated prompt differs from previous ones in that it contains several examples of the Question > Thought > Action > Observation loop, that include the Search and Lookup tools. [For more information about this approach, take a look at the React paper of Yao et al. (2023)](https://arxiv.org/pdf/2210.03629.pdf).

In [44]:
print(docstore_agent.agent.llm_chain.prompt.template)



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United St

In [43]:
count_tokens(docstore_agent, "What were Archimedes' creations?")



> Entering new AgentExecutor chain...
Thought: I need to search Archimedes and find his creations.
Action: Search[Archimedes]
Observation: Archimedes of Syracuse (; c. 287 – c. 212 BC) was a Greek mathematician, physicist, engineer, astronomer, and inventor from the ancient city of Syracuse in Sicily. Although few details of his life are known, he is regarded as one of the leading scientists in classical antiquity. Considered the greatest mathematician of ancient history, and one of the greatest of all time, Archimedes anticipated modern calculus and analysis by applying the concept of the infinitely small and the method of exhaustion to derive and rigorously prove a range of geometrical theorems. These include the area of a circle, the surface area and volume of a sphere, the area of an ellipse, the area under a parabola, the volume of a segment of a paraboloid of revolution, the volume of a segment of a hyperboloid of revolution, and the area of a spiral.Archimedes' other mathemati

{'input': "What were Archimedes' creations?",
 'output': "deriving an approximation of pi, defining and investigating the Archimedean spiral, devising a system using exponentiation for expressing very large numbers, proof of the law of the lever, the widespread use of the concept of center of gravity, the enunciation of the law of buoyancy or Archimedes' principle, his screw pump, compound pulleys, and defensive war machines"}

### 3.4 - Self-ask with search

This is the first-choice agent to use when using LLM's to extract information with a search engine. The agent will ask follow-up questions and use the search functionality to get intermediate answers that help it get to a final answer.

We will use the DuckDuckGo search engine because it is free and does not require an API key.

In [48]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='duckduckgo search'
    )
]

self_ask_with_search = initialize_agent(
    tools, 
    language_model, 
    agent="self-ask-with-search",
    verbose=True, 
    max_iterations=3
)

As we can see below, the prompt is basically a series of many examples to show the LLM how to ask follow up questions to a search tool until it can get to the final answer.

In [46]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [52]:
# question = "What is the hometown of the current men's U.S. Open champion?"
question = "What is the name of current US president's wife?"
result = count_tokens(
    self_ask_with_search,
    question
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: Who is the current US president?
Intermediate answer: U.S. facts and figures Presidents, vice presidents, and first ladies Presidents, vice presidents, and first ladies Learn about the duties of president, vice president, and first lady of the United States. Find out how to contact and learn more about current and past leaders. President of the United States Vice president of the United States Joe Biden, byname of Joseph Robinette Biden, Jr., (born November 20, 1942, Scranton, Pennsylvania, U.S.), 46th president of the United States (2021- ) and 47th vice president of the United States (2009-17) in the Democratic administration of Pres. Barack Obama. He previously represented Delaware in the U.S. Senate (1973-2009). President Joe Biden has formally announced he's seeking reelection As the head of the government of the United States, the president is arguably the most powerful government official in the world. The president is ele

This one is tricky because it depends on the search engine results. It can get the correct answer on several steps and not be able to properly follow them, and the chain is stopped.

## 4 - "Plan-and-Execute" (PaE) agents

"Plan-and-Execute" (PaE) agents accomplish an objective by first planning what to do, then executing the sub tasks. This idea is largely inspired by [BabyAGI](https://github.com/yoheinakajima/babyagi) and then the ["Plan-and-Solve" (Wang et al., 2023) paper](https://arxiv.org/pdf/2305.04091.pdf).
* The planning is almost always done by an LLM.
* The execution is usually done by an action agent (equipped with tools).

**High level pseudocode of PaE agents:**
* Some user input is received
* The **planner** lists out the steps to take
* The **executor** goes through the list of steps, executing them

**Typical way to construct a PaE agent:**
* [**Planner:**](https://python.langchain.com/en/latest/modules/agents/plan_and_execute.html#planner-executor-and-agent) usually a chat agent. It can be invoked using the `load_chat_planner(model)` function.
    * Language Model
* [**Executor:**](https://python.langchain.com/en/latest/modules/agents/plan_and_execute.html#planner-executor-and-agent) an agent executor, which takes an agent to decide which tools to call and in what order. It can be invoked with the `load_agent_executor(model, tools)` function.
    * Language Model
    * Toolkit
* [**Agent:**](https://python.langchain.com/en/latest/modules/agents/plan_and_execute.html#planner-executor-and-agent) the PaE agent. It can be invoked with the `PlanAndExecute(planner=planner, executer=executor)` class.


In [53]:
from langchain import LLMMathChain

search = DuckDuckGoSearchRun()
llm_math_chain = LLMMathChain.from_llm(llm=language_model, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

In [57]:
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

planner = load_chat_planner(language_model)
executor = load_agent_executor(language_model, tools, verbose=True)
pae_agent = PlanAndExecute(planner=planner, executer=executor, verbose=True)

In [61]:
pae_agent

PlanAndExecute(memory=None, callbacks=None, callback_manager=None, verbose=True, planner=LLMPlanner(llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=ChatPromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, messages=[SystemMessage(content="Let's first understand the problem and devise a plan to solve the problem. Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. Please make the plan the minimum number of steps required to accurately complete the task. If the task is a question, the final step should almost always be 'Given the above steps taken, please respond to the users original question'. At the end of your plan, say '<END_OF_PLAN>'", additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='{input}', template_format='f-string', validate_template=True), additional_k

In [58]:
pae_agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new PlanAndExecute chain...
steps=[Step(value="Research Leo DiCaprio's current girlfriend"), Step(value='Find her current age'), Step(value='Raise her age to the 0.43 power'), Step(value="Given the above steps taken, please respond to the user's original question\n")]

> Entering new AgentExecutor chain...

Action:
```
{
  "action": "Search",
  "action_input": "Leo DiCaprio current girlfriend"
}
```


Observation: January 2018: Leo and Camila have a snowy vacation in Colorado. A month after they were first spotted together, the couple vacationed together in Aspen. Fellow actor and friend Tobey Maguire was... DiCaprio, 48, has long been the butt of jokes about his tendency to date women no older than 25 and now he's rumored to be dating 19-year-old model, Eden Polani. The alleged new couple were... "Titanic" star Leonardo DiCaprio, 48, was trolled online this week over an alleged romance with 19-year-old model Eden Polani. DiCaprio, who has been dragged in the past for only

"Based on the information gathered from the research, Leonardo DiCaprio's current girlfriend Eden Polani is 19 years old and her age raised to the 0.43 power is 3.55"